In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import numpy as np

from G1 import g1
from G2 import g2

## FIFAランキングスコア

In [2]:
url = "https://www.fifa.com/fifa-world-ranking/ranking-table/men/index.html"
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}
request = Request(url=url, headers=headers)
response = urlopen(request)
fifa = BeautifulSoup(response, 'html.parser')

In [3]:
for t in fifa.find_all("table"):
    if t.get("class") == "table tbl-ranking table-striped".split():
        f_table = t

In [4]:
f_body = f_table.find("tbody")

In [5]:
f_body.find_all("tr")[0].get("ranking-rawpoints")

In [6]:
f_list = []
f_scores = []
for tr in f_body.find_all("tr"):
    if not re.match(r"rnk_*", tr.get("id")):
        continue
    for td in tr.find_all("td"):
        if td.get("class") == ["tbl-teamname"]:
            f_list.append(td.text)
        if td.get("class") == ["tbl-points"]:
            f_scores.append(float(td.text.split("(")[-1][:-1]))

In [7]:
len(f_list) == len(f_scores)

True

In [8]:
print("チーム数:", len(f_list))

チーム数: 211


## ポケモン種族値

In [9]:
# 第１世代のポケモンのリスト
p_g1 = list(map(lambda x: x.split()[-1], g1.split("\n")))
p_g2 = list(map(lambda x: x.split()[-1], g2.split("\n")))

In [10]:
response = urlopen("https://pente.koro-pokemon.com/data/ranking-total.shtml")
poke = BeautifulSoup(response, 'html.parser')

In [11]:
table = poke.find_all("table")

In [12]:
for t in table:
    if " ".join(t.get("class")) == "ta4 ta6":
        target = t
        break

In [28]:
p_list = []
p_scores = []
for tr in target.find_all("tr"):
    p_line = []
    for li in tr.find_all("li"):
        if li.text in p_g1 or li.text in p_g2:
            p_line.append(li.text)
    if p_line != []:
        p_list.append(p_line)
        th = tr.find("th")
        assert th.get("class") == ["th2"]
        p_scores.append(float(th.text))

In [29]:
len(p_scores) == len(p_list)

True

In [30]:
print("種族値の種類数:", len(p_scores))

種族値の種類数: 61


In [31]:
all_p_list = np.concatenate(p_list)

In [32]:
len(all_p_list)

192

## 正規化なし(第1,2世代のみ)

In [36]:
for i, (p_name, p_score) in enumerate(zip(p_list, p_scores)):
    print(f_list[i], f_scores[i], p_name, p_score)

Germany 1558.49 ['ミュウツー', 'ルギア', 'ホウオウ', 'レックウザ'] 680.0
Brazil 1430.57 ['ケッキング', 'カイオーガ', 'グラードン'] 670.0
Belgium 1298.32 ['カイリュー', 'バンギラス', 'ボーマンダ', 'メタグロス', 'ラティアス', 'ラティオス', 'ミュウ', 'セレビィ', 'ジラーチ'] 600.0
Portugal 1273.69 ['フリーザー', 'サンダー', 'ファイヤー', 'ライコウ', 'エンテイ', 'スイクン', 'レジロック', 'レジアイス', 'レジスチル'] 580.0
Argentina 1241.45 ['ウインディ'] 555.0
Switzerland 1198.72 ['ギャラドス', 'カビゴン', 'キングドラ', 'ハピナス', 'ミロカロス'] 540.0
France 1198.13 ['ラプラス', 'クロバット', 'ラグラージ'] 535.0
Poland 1182.61 ['リザードン', 'バクフーン'] 534.0
Chile 1134.66 ['カメックス', 'オーダイル', 'ジュカイン', 'バシャーモ', 'トドゼルガ', 'ボスゴドラ'] 530.0
Spain 1125.5 ['フシギバナ', 'パルシェン', 'シャワーズ', 'サンダース', 'ブースター', 'メガニウム', 'エーフィ', 'ブラッキー'] 525.0
Peru 1125.39 ['スターミー', 'フライゴン'] 520.0
Denmark 1051.44 ['サーナイト'] 518.0
England 1050.84 ['ドククラゲ', 'プテラ', 'ポリゴン2'] 515.0
Uruguay 1018.41 ['ニョロボン', 'デンリュウ', 'ハガネール'] 510.0
Mexico 989.44 ['ニドクイン', 'ニドキング', 'カイリキー', 'ツボツボ'] 505.0
Colombia 985.97 ['ゴルダック', 'フーディン', 'ギャロップ', 'ゲンガー', 'ストライク', 'カイロス', 'ニョロトノ', 'ハッサム', 'ヘラクロス', 'リングマ', 'ヘルガー', '

## 正規化あり(第1,2世代のみ)

In [34]:
norm_p_scores = (np.array(p_scores) - np.mean(p_scores)) / np.std(p_scores)
norm_f_scores = (np.array(f_scores) - np.mean(f_scores)) / np.std(f_scores)

In [35]:
print("FIFA\tPokemon")
for n_f_score, f_score, f_team in zip(norm_f_scores, f_scores, f_list):
    pos = np.argmin(np.abs(norm_p_scores - n_f_score))
    print("{} (score:{}, n_score:{:.4f})\t{} (score:{}, n_score{:.4f})".format(
        f_team, f_score, n_f_score, p_list[pos][0], p_scores[pos], norm_p_scores[pos]))

FIFA	Pokemon
Germany (score:1558.49, n_score:3.4787)	ミュウツー (score:680.0, n_score2.6846)
Brazil (score:1430.57, n_score:3.0957)	ミュウツー (score:680.0, n_score2.6846)
Belgium (score:1298.32, n_score:2.6997)	ミュウツー (score:680.0, n_score2.6846)
Portugal (score:1273.69, n_score:2.6259)	ミュウツー (score:680.0, n_score2.6846)
Argentina (score:1241.45, n_score:2.5294)	ケッキング (score:670.0, n_score2.5672)
Switzerland (score:1198.72, n_score:2.4014)	ケッキング (score:670.0, n_score2.5672)
France (score:1198.13, n_score:2.3996)	ケッキング (score:670.0, n_score2.5672)
Poland (score:1182.61, n_score:2.3532)	ケッキング (score:670.0, n_score2.5672)
Chile (score:1134.66, n_score:2.2096)	ケッキング (score:670.0, n_score2.5672)
Spain (score:1125.5, n_score:2.1821)	ケッキング (score:670.0, n_score2.5672)
Peru (score:1125.39, n_score:2.1818)	ケッキング (score:670.0, n_score2.5672)
Denmark (score:1051.44, n_score:1.9604)	カイリュー (score:600.0, n_score1.7453)
England (score:1050.84, n_score:1.9586)	カイリュー (score:600.0, n_score1.7453)
Uruguay (score:1